In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_FV
from utils_training import FVTrainingData
from utils_training import evaluate_FVmodel

In [4]:
from behavior_model import FVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 4, 0.25
acc_std_dev = 0.07
Inputs, Inputs_Vl, Outputs, Weights = FVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('Idx05_train_val_test.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Inputs_Vl_train = [Inputs_Vl[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Inputs_Vl_val = [Inputs_Vl[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_FV(Inputs_train, Inputs_Vl_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_FV(Inputs_val, Inputs_Vl_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = FVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_fv_val_05.pth'
best_train_model_path = f'.\\weights\\bc_fv_train_05.pth'
best_trainval_model_path = f'.\\weights\\bc_fv_trainval_05.pth'

In [13]:
num_epochs = 2000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, inputs_vl, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        inputs_vl = inputs_vl.reshape(inputs_vl.shape[0],1,inputs_vl.shape[1])
        inputs_vl = inputs_vl.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs,inputs_vl)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_FVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_FVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/2000, Loss_mean: 361.65,      Loss_min: 359.57, Loss_max: 366.46 
      Loss_mean_val: 360.01, Loss_min_val: 346.95,      Loss_max_val: 369.96 
      Loss_mean_trainval: 360.17
Epoch 2/2000, Loss_mean: 342.44,      Loss_min: 335.21, Loss_max: 348.58 
      Loss_mean_val: 338.81, Loss_min_val: 298.4,      Loss_max_val: 367.8 
      Loss_mean_trainval: 339.15
Epoch 3/2000, Loss_mean: 325.9,      Loss_min: 316.59, Loss_max: 330.97 
      Loss_mean_val: 320.07, Loss_min_val: 261.9,      Loss_max_val: 344.66 
      Loss_mean_trainval: 320.62
Epoch 4/2000, Loss_mean: 312.56,      Loss_min: 297.29, Loss_max: 328.91 
      Loss_mean_val: 308.83, Loss_min_val: 243.59,      Loss_max_val: 343.02 
      Loss_mean_trainval: 309.18
Epoch 5/2000, Loss_mean: 303.63,      Loss_min: 290.51, Loss_max: 311.25 
      Loss_mean_val: 298.55, Loss_min_val: 249.45,      Loss_max_val: 340.62 
      Loss_mean_trainval: 299.03
Epoch 6/2000, Loss_mean: 294.82,      Loss_min: 276.78, Loss_max: 311.88 
     

Epoch 46/2000, Loss_mean: 134.68,      Loss_min: 129.13, Loss_max: 145.74 
      Loss_mean_val: 148.21, Loss_min_val: 100.98,      Loss_max_val: 208.69 
      Loss_mean_trainval: 146.92
Epoch 47/2000, Loss_mean: 139.38,      Loss_min: 105.5, Loss_max: 164.74 
      Loss_mean_val: 145.15, Loss_min_val: 105.01,      Loss_max_val: 214.27 
      Loss_mean_trainval: 144.6
Epoch 48/2000, Loss_mean: 137.17,      Loss_min: 108.25, Loss_max: 160.47 
      Loss_mean_val: 145.31, Loss_min_val: 111.41,      Loss_max_val: 207.95 
      Loss_mean_trainval: 144.54
Epoch 49/2000, Loss_mean: 133.81,      Loss_min: 119.95, Loss_max: 156.42 
      Loss_mean_val: 143.73, Loss_min_val: 79.9,      Loss_max_val: 213.34 
      Loss_mean_trainval: 142.78
Epoch 50/2000, Loss_mean: 133.22,      Loss_min: 113.4, Loss_max: 164.01 
      Loss_mean_val: 145.08, Loss_min_val: 101.55,      Loss_max_val: 193.75 
      Loss_mean_trainval: 143.95
Epoch 51/2000, Loss_mean: 136.42,      Loss_min: 105.58, Loss_max: 166.9 
 

Epoch 91/2000, Loss_mean: 106.11,      Loss_min: 89.79, Loss_max: 119.84 
      Loss_mean_val: 124.33, Loss_min_val: 86.64,      Loss_max_val: 173.29 
      Loss_mean_trainval: 122.59
Epoch 92/2000, Loss_mean: 104.9,      Loss_min: 86.78, Loss_max: 118.19 
      Loss_mean_val: 124.25, Loss_min_val: 85.94,      Loss_max_val: 178.22 
      Loss_mean_trainval: 122.4
Epoch 93/2000, Loss_mean: 107.49,      Loss_min: 84.74, Loss_max: 127.97 
      Loss_mean_val: 122.26, Loss_min_val: 84.34,      Loss_max_val: 164.52 
      Loss_mean_trainval: 120.86
Epoch 94/2000, Loss_mean: 105.92,      Loss_min: 82.91, Loss_max: 124.63 
      Loss_mean_val: 122.69, Loss_min_val: 71.45,      Loss_max_val: 162.56 
      Loss_mean_trainval: 121.09
Epoch 95/2000, Loss_mean: 104.44,      Loss_min: 91.8, Loss_max: 120.0 
      Loss_mean_val: 122.07, Loss_min_val: 74.45,      Loss_max_val: 175.95 
      Loss_mean_trainval: 120.39
Epoch 96/2000, Loss_mean: 103.12,      Loss_min: 94.54, Loss_max: 118.55 
      Loss

Epoch 136/2000, Loss_mean: 88.09,      Loss_min: 70.58, Loss_max: 108.82 
      Loss_mean_val: 110.3, Loss_min_val: 70.13,      Loss_max_val: 213.76 
      Loss_mean_trainval: 108.18
Epoch 137/2000, Loss_mean: 87.48,      Loss_min: 67.64, Loss_max: 112.05 
      Loss_mean_val: 108.97, Loss_min_val: 59.29,      Loss_max_val: 165.55 
      Loss_mean_trainval: 106.92
Epoch 138/2000, Loss_mean: 87.46,      Loss_min: 77.82, Loss_max: 94.71 
      Loss_mean_val: 107.46, Loss_min_val: 70.22,      Loss_max_val: 168.51 
      Loss_mean_trainval: 105.55
Epoch 139/2000, Loss_mean: 93.2,      Loss_min: 58.4, Loss_max: 115.11 
      Loss_mean_val: 106.07, Loss_min_val: 66.8,      Loss_max_val: 166.6 
      Loss_mean_trainval: 104.84
Epoch 140/2000, Loss_mean: 88.26,      Loss_min: 74.78, Loss_max: 102.07 
      Loss_mean_val: 108.52, Loss_min_val: 55.31,      Loss_max_val: 154.08 
      Loss_mean_trainval: 106.58
Epoch 141/2000, Loss_mean: 87.31,      Loss_min: 63.93, Loss_max: 101.61 
      Loss_m

Epoch 181/2000, Loss_mean: 74.7,      Loss_min: 70.29, Loss_max: 77.69 
      Loss_mean_val: 98.87, Loss_min_val: 49.41,      Loss_max_val: 147.56 
      Loss_mean_trainval: 96.56
Epoch 182/2000, Loss_mean: 75.69,      Loss_min: 64.35, Loss_max: 99.56 
      Loss_mean_val: 98.31, Loss_min_val: 45.46,      Loss_max_val: 154.99 
      Loss_mean_trainval: 96.15
Epoch 183/2000, Loss_mean: 74.25,      Loss_min: 47.18, Loss_max: 92.43 
      Loss_mean_val: 96.43, Loss_min_val: 64.6,      Loss_max_val: 139.76 
      Loss_mean_trainval: 94.31
Epoch 184/2000, Loss_mean: 74.84,      Loss_min: 55.58, Loss_max: 95.38 
      Loss_mean_val: 97.66, Loss_min_val: 36.5,      Loss_max_val: 134.3 
      Loss_mean_trainval: 95.49
Epoch 185/2000, Loss_mean: 73.39,      Loss_min: 64.24, Loss_max: 83.77 
      Loss_mean_val: 96.78, Loss_min_val: 59.48,      Loss_max_val: 149.54 
      Loss_mean_trainval: 94.55
Epoch 186/2000, Loss_mean: 75.02,      Loss_min: 53.99, Loss_max: 85.95 
      Loss_mean_val: 95.49

...

Epoch 1791/2000, Loss_mean: 10.02,      Loss_min: 7.04, Loss_max: 12.18 
      Loss_mean_val: 73.0, Loss_min_val: 36.13,      Loss_max_val: 149.37 
      Loss_mean_trainval: 67.0
Epoch 1792/2000, Loss_mean: 10.24,      Loss_min: 8.32, Loss_max: 11.37 
      Loss_mean_val: 73.05, Loss_min_val: 6.83,      Loss_max_val: 147.24 
      Loss_mean_trainval: 67.06
Epoch 1793/2000, Loss_mean: 10.03,      Loss_min: 8.25, Loss_max: 11.66 
      Loss_mean_val: 70.57, Loss_min_val: 38.02,      Loss_max_val: 152.62 
      Loss_mean_trainval: 64.79
Epoch 1794/2000, Loss_mean: 10.31,      Loss_min: 9.31, Loss_max: 12.44 
      Loss_mean_val: 74.12, Loss_min_val: 42.61,      Loss_max_val: 119.95 
      Loss_mean_trainval: 68.04
Epoch 1795/2000, Loss_mean: 10.53,      Loss_min: 7.45, Loss_max: 12.51 
      Loss_mean_val: 73.78, Loss_min_val: 37.92,      Loss_max_val: 161.36 
      Loss_mean_trainval: 67.75
Epoch 1796/2000, Loss_mean: 10.31,      Loss_min: 9.43, Loss_max: 11.22 
      Loss_mean_val: 73.2

Epoch 1837/2000, Loss_mean: 10.1,      Loss_min: 8.76, Loss_max: 11.1 
      Loss_mean_val: 73.96, Loss_min_val: 32.48,      Loss_max_val: 150.1 
      Loss_mean_trainval: 67.87
Epoch 1838/2000, Loss_mean: 9.9,      Loss_min: 7.62, Loss_max: 11.44 
      Loss_mean_val: 72.19, Loss_min_val: 39.53,      Loss_max_val: 126.17 
      Loss_mean_trainval: 66.25
Epoch 1839/2000, Loss_mean: 9.68,      Loss_min: 7.81, Loss_max: 11.23 
      Loss_mean_val: 76.57, Loss_min_val: 27.75,      Loss_max_val: 205.15 
      Loss_mean_trainval: 70.19
Epoch 1840/2000, Loss_mean: 9.85,      Loss_min: 7.69, Loss_max: 11.08 
      Loss_mean_val: 72.48, Loss_min_val: 27.55,      Loss_max_val: 124.8 
      Loss_mean_trainval: 66.5
Epoch 1841/2000, Loss_mean: 9.69,      Loss_min: 8.45, Loss_max: 10.92 
      Loss_mean_val: 71.85, Loss_min_val: 32.92,      Loss_max_val: 138.39 
      Loss_mean_trainval: 65.92
Epoch 1842/2000, Loss_mean: 9.67,      Loss_min: 7.97, Loss_max: 11.56 
      Loss_mean_val: 73.54, Loss_

Epoch 1883/2000, Loss_mean: 9.39,      Loss_min: 8.22, Loss_max: 10.56 
      Loss_mean_val: 76.08, Loss_min_val: 29.53,      Loss_max_val: 141.02 
      Loss_mean_trainval: 69.72
Epoch 1884/2000, Loss_mean: 9.45,      Loss_min: 7.48, Loss_max: 11.32 
      Loss_mean_val: 74.08, Loss_min_val: 32.66,      Loss_max_val: 129.71 
      Loss_mean_trainval: 67.91
Epoch 1885/2000, Loss_mean: 9.58,      Loss_min: 6.29, Loss_max: 11.91 
      Loss_mean_val: 76.04, Loss_min_val: 29.92,      Loss_max_val: 148.86 
      Loss_mean_trainval: 69.7
Epoch 1886/2000, Loss_mean: 9.84,      Loss_min: 6.52, Loss_max: 12.8 
      Loss_mean_val: 77.05, Loss_min_val: 20.97,      Loss_max_val: 140.82 
      Loss_mean_trainval: 70.64
Epoch 1887/2000, Loss_mean: 9.55,      Loss_min: 7.72, Loss_max: 12.43 
      Loss_mean_val: 74.08, Loss_min_val: 24.93,      Loss_max_val: 182.04 
      Loss_mean_trainval: 67.92
Epoch 1888/2000, Loss_mean: 9.26,      Loss_min: 7.15, Loss_max: 11.44 
      Loss_mean_val: 74.0, Los

Epoch 1929/2000, Loss_mean: 9.26,      Loss_min: 7.37, Loss_max: 10.95 
      Loss_mean_val: 74.81, Loss_min_val: 27.84,      Loss_max_val: 145.39 
      Loss_mean_trainval: 68.55
Epoch 1930/2000, Loss_mean: 9.06,      Loss_min: 6.92, Loss_max: 10.25 
      Loss_mean_val: 73.36, Loss_min_val: 33.97,      Loss_max_val: 142.77 
      Loss_mean_trainval: 67.23
Epoch 1931/2000, Loss_mean: 9.03,      Loss_min: 7.9, Loss_max: 10.3 
      Loss_mean_val: 73.36, Loss_min_val: 21.54,      Loss_max_val: 125.96 
      Loss_mean_trainval: 67.23
Epoch 1932/2000, Loss_mean: 9.27,      Loss_min: 8.15, Loss_max: 11.43 
      Loss_mean_val: 75.81, Loss_min_val: 36.75,      Loss_max_val: 135.09 
      Loss_mean_trainval: 69.46
Epoch 1933/2000, Loss_mean: 9.04,      Loss_min: 8.29, Loss_max: 10.45 
      Loss_mean_val: 75.26, Loss_min_val: 36.57,      Loss_max_val: 111.1 
      Loss_mean_trainval: 68.95
Epoch 1934/2000, Loss_mean: 9.02,      Loss_min: 7.09, Loss_max: 11.1 
      Loss_mean_val: 73.88, Loss

Epoch 1975/2000, Loss_mean: 9.07,      Loss_min: 6.8, Loss_max: 10.94 
      Loss_mean_val: 76.74, Loss_min_val: 30.76,      Loss_max_val: 156.16 
      Loss_mean_trainval: 70.28
Epoch 1976/2000, Loss_mean: 8.89,      Loss_min: 7.3, Loss_max: 10.67 
      Loss_mean_val: 74.58, Loss_min_val: 41.31,      Loss_max_val: 129.19 
      Loss_mean_trainval: 68.32
Epoch 1977/2000, Loss_mean: 8.78,      Loss_min: 7.21, Loss_max: 10.49 
      Loss_mean_val: 74.93, Loss_min_val: 27.16,      Loss_max_val: 140.18 
      Loss_mean_trainval: 68.62
Epoch 1978/2000, Loss_mean: 8.97,      Loss_min: 7.11, Loss_max: 12.75 
      Loss_mean_val: 74.18, Loss_min_val: 37.21,      Loss_max_val: 141.23 
      Loss_mean_trainval: 67.96
Epoch 1979/2000, Loss_mean: 8.93,      Loss_min: 6.43, Loss_max: 12.0 
      Loss_mean_val: 75.04, Loss_min_val: 33.26,      Loss_max_val: 127.38 
      Loss_mean_trainval: 68.73
Epoch 1980/2000, Loss_mean: 8.9,      Loss_min: 6.95, Loss_max: 10.38 
      Loss_mean_val: 75.25, Loss